<a href="https://colab.research.google.com/github/tubagokhan/ADGM/blob/main/ParsingIntermediateFormat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import random
import string

def generate_unique_identifier(used_identifiers):
    while True:
        # Generate an identifier consisting of 3 letters followed by 3 digits
        identifier = ''.join(random.choices(string.ascii_letters, k=3)) + ''.join(random.choices(string.digits, k=3))
        # Check if the identifier is unique
        if identifier not in used_identifiers:
            return identifier

def process_file(input_file_path, output_file_path):
    data = []
    current_entry = None
    inside_table = False  # Variable to track if we're inside a table
    used_identifiers = set()  # Set to keep track of used unique identifiers

    with open(input_file_path, 'r') as file:
        for line in file:
            if '/Table Start' in line:
                inside_table = True
                current_entry['Text'] += '\n' + line.strip()
                continue
            elif '/Table End' in line:
                inside_table = False
                current_entry['Text'] += '\n' + line.strip()
                continue

            if inside_table:
                current_entry['Text'] += '\n' + line.strip()
            elif line.startswith('PART ') or line[0].isdigit():
                parts = line.split('\t', 1)
                context_id = parts[0].strip()
                text = parts[1].strip() if len(parts) > 1 else ''

                if current_entry:
                    data.append(current_entry)

                # Generate a unique identifier for the new entry
                unique_id = generate_unique_identifier(used_identifiers)
                used_identifiers.add(unique_id)

                current_entry = {'ID': unique_id, 'ContextID': context_id, 'Text': text}
            elif current_entry:
                current_entry['Text'] += '\n' + line.strip()

    if current_entry:
        data.append(current_entry)

    with open(output_file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Replace 'input.txt' and 'output.json' with your actual file paths
process_file('/content/COBS_VER15.150823.txt', '/content/COBS_VER15.150823.json')
